<a href="https://colab.research.google.com/github/avocadopelvis/nagamese-english/blob/main/mbart-50-fine-tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already 

In [2]:
# !git config --global user.email ""
# !git config --global user.name ""

from huggingface_hub import notebook_login
notebook_login()

### Load the data

In [6]:
from datasets import load_dataset

raw_datasets = load_dataset("japanese-denim/naga-eng")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['English', 'Nagamese'],
        num_rows: 7950
    })
})

In [7]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.8, seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['English', 'Nagamese'],
        num_rows: 6360
    })
    test: Dataset({
        features: ['English', 'Nagamese'],
        num_rows: 1590
    })
})

In [ ]:
split_datasets["validation"] = split_datasets.pop("test")

In [ ]:
split_datasets["train"][1]

{'English': 'Jesus answered and said to them, "Even if I testify about myself, my testimony is true. I know where I came from and where I am going, but you do not know where I came from or where I am going.',
 'Nagamese': 'Jisu taikhan ke jowab di koise, “Jodi Moi nijor karone sakhi dile bi, Moi laga sakhi hosa ase. Kilemane Moi kot pora ahise aru kot te jabole ase itu Moi jane, kintu tumi khan to Moi kot pora ahise aru kot te jabole ase najane.'}

### Data Processing

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "facebook/mbart-large-50"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, src_lang="en_XX", tgt_lang="ng_XX")

In [ ]:
en_sentence = split_datasets["train"][1]["English"]
ng_sentence = split_datasets["train"][1]["Nagamese"]

inputs = tokenizer(en_sentence, text_target=ng_sentence)
inputs

{'input_ids': [250004, 9716, 194663, 136, 2804, 47, 2856, 4, 44, 647, 1353, 2174, 87, 3034, 40383, 1672, 35978, 4, 759, 3034, 2414, 299, 83, 29568, 5, 87, 3714, 7440, 87, 21449, 1295, 136, 7440, 87, 444, 7730, 4, 1284, 398, 54, 959, 3714, 7440, 87, 21449, 1295, 707, 7440, 87, 444, 7730, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [3, 33443, 34, 1013, 124880, 311, 741, 72947, 45, 298, 3075, 4, 52, 16604, 428, 32129, 300, 15503, 12208, 86, 57, 52568, 26052, 333, 4, 32129, 6327, 57, 52568, 3080, 11, 10, 184, 5, 1519, 41092, 86, 32129, 1975, 196, 11, 1263, 3075, 72205, 1975, 120, 79, 61217, 10, 184, 752, 32129, 47363, 4, 200, 18461, 111788, 70144, 47, 32129, 1975, 196, 11, 1263, 3075, 72205, 1975, 120, 79, 61217, 10, 184, 2034, 6130, 5, 2]}

In [ ]:
max_length = 128

def preprocess_function(examples):
    inputs = [ex for ex in examples["English"]]
    targets = [ex for ex in examples["Nagamese"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )

    return model_inputs

tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Map:   0%|          | 0/6360 [00:00<?, ? examples/s]

Map:   0%|          | 0/1590 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model_checkpoint = "facebook/mbart-large-50"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

You're using a MBart50TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [ ]:
batch["labels"]

tensor([[     3,  33443,     34,   1013, 124880,    311,    741,  72947,     45,
            298,   3075,      4,     52,  16604,    428,  32129,    300,  15503,
          12208,     86,     57,  52568,  26052,    333,      4,  32129,   6327,
             57,  52568,   3080,     11,     10,    184,      5,   1519,  41092,
             86,  32129,   1975,    196,     11,   1263,   3075,  72205,   1975,
            120,     79,  61217,     10,    184,    752,  32129,  47363,      4,
            200,  18461, 111788,  70144,     47,  32129,   1975,    196,     11,
           1263,   3075,  72205,   1975,    120,     79,  61217,     10,    184,
           2034,   6130,      5,      2],
        [     3,  10208,  72205,   1013,   6327,     57,   4450,  70144, 185709,
             53,   7366,  72205,  30733,   4526,   6327,  86864,  40681,    133,
            764,     79,   3075,      4,    200,  41092,     86,    663,   3137,
            516,   1913,    192,    196,     11,   1013, 124880,   

In [ ]:
batch["decoder_input_ids"]

tensor([[     2,      3,  33443,     34,   1013, 124880,    311,    741,  72947,
             45,    298,   3075,      4,     52,  16604,    428,  32129,    300,
          15503,  12208,     86,     57,  52568,  26052,    333,      4,  32129,
           6327,     57,  52568,   3080,     11,     10,    184,      5,   1519,
          41092,     86,  32129,   1975,    196,     11,   1263,   3075,  72205,
           1975,    120,     79,  61217,     10,    184,    752,  32129,  47363,
              4,    200,  18461, 111788,  70144,     47,  32129,   1975,    196,
             11,   1263,   3075,  72205,   1975,    120,     79,  61217,     10,
            184,   2034,   6130,      5],
        [     2,      3,  10208,  72205,   1013,   6327,     57,   4450,  70144,
         185709,     53,   7366,  72205,  30733,   4526,   6327,  86864,  40681,
            133,    764,     79,   3075,      4,    200,  41092,     86,    663,
           3137,    516,   1913,    192,    196,     11,   1013, 12

In [ ]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[3, 33443, 34, 1013, 124880, 311, 741, 72947, 45, 298, 3075, 4, 52, 16604, 428, 32129, 300, 15503, 12208, 86, 57, 52568, 26052, 333, 4, 32129, 6327, 57, 52568, 3080, 11, 10, 184, 5, 1519, 41092, 86, 32129, 1975, 196, 11, 1263, 3075, 72205, 1975, 120, 79, 61217, 10, 184, 752, 32129, 47363, 4, 200, 18461, 111788, 70144, 47, 32129, 1975, 196, 11, 1263, 3075, 72205, 1975, 120, 79, 61217, 10, 184, 2034, 6130, 5, 2]
[3, 10208, 72205, 1013, 6327, 57, 4450, 70144, 185709, 53, 7366, 72205, 30733, 4526, 6327, 86864, 40681, 133, 764, 79, 3075, 4, 200, 41092, 86, 663, 3137, 516, 1913, 192, 196, 11, 1013, 124880, 311, 14237, 120, 502, 9254, 74473, 61217, 3513, 107760, 13, 5, 2]


In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

### Fine-tuning the model

In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"mbart-50-finetuned-eng-to-naga",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.evaluate(max_length=max_length)

{'eval_loss': 10.497693061828613,
 'eval_bleu': 0.20099852753763722,
 'eval_runtime': 269.1904,
 'eval_samples_per_second': 5.907,
 'eval_steps_per_second': 0.371}

In [ ]:
trainer.train()

Step,Training Loss
500,5.499600
1000,5.046700
1500,2.760400
2000,2.475100
2500,3.958200
3000,4.225300
3500,2.409000
4000,2.079900
4500,1.942800
5000,1.945700


TrainOutput(global_step=19080, training_loss=1.8274492467724301, metrics={'train_runtime': 8524.2798, 'train_samples_per_second': 2.238, 'train_steps_per_second': 2.238, 'total_flos': 1306303074091008.0, 'train_loss': 1.8274492467724301, 'epoch': 3.0})

In [ ]:
trainer.evaluate(max_length=max_length)

{'eval_loss': 1.4960336685180664,
 'eval_bleu': 20.9247827422808,
 'eval_runtime': 2709.558,
 'eval_samples_per_second': 0.587,
 'eval_steps_per_second': 0.587,
 'epoch': 3.0}

In [ ]:
trainer.push_to_hub(tags="translation", commit_message="Training complete")

'https://huggingface.co/japanese-denim/mbart-50-finetuned-eng-to-naga/tree/main/'

In [3]:
from transformers import pipeline

model_checkpoint = "japanese-denim/mbart-50-finetuned-eng-to-naga"
translator = pipeline("translation", model=model_checkpoint, src_lang="en_XX", tgt_lang="ng_XX")

In [4]:
translator("The boat was in the sea")

[{'translation_text': 'Aru naw to samundar te thakise,'}]

In [ ]:
translator(
    "When he was walking beside the Sea of Galilee, he saw Simon and Andrew the brother of Simon casting a net in the sea, for they were fishermen."
)

[{'translation_text': 'Aru jitia Tai Galilee laga samundar kinar te berai thakise, Tai Simon aru tai laga bhai Andrew ke samundar te jal phelai thaka dikhise, kilemane tai khan maas dhora manu thakise.'}]

In [ ]:
translator("They were eating fish")

[{'translation_text': 'Aru taikhan maas khai thakise.'}]

In [9]:
split_datasets["test"][7]

{'English': 'Seeing their faith, Jesus said, "Man, your sins are forgiven you."',
 'Nagamese': 'Taikhan laga biswas dikhi kena, Jisu koise, “Manu, tumi laga paapkhan maph hoise.”'}

In [10]:
translator(split_datasets["test"][7]["English"])

[{'translation_text': 'Titia Jisu taikhan laga biswas dikhi kena koise, “Manu, tumi laga paap khan sob maph koridise.”'}]